In [122]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Assignment 3.xls')

In [5]:
df.shape

(100, 25)

In [6]:
df.head(2)

,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
11: Star Wars: Episode IV - A New Hope (1977),NaN,4.5,5.0,4.5,4.0,4.0,NaN,5.0,4.0,5.0,...,4.0,NaN,4.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN
12: Finding Nemo (2003),NaN,5.0,5.0,NaN,4.0,4.0,4.5,4.5,4.0,5.0,...,4.0,NaN,3.5,4.0,2.0,3.5,NaN,NaN,NaN,3.5


# Part 1 - Without Normalization

## Complete the user-by-user correlations matrix.

In [10]:
corr_df = df.corr()

To check your math, note that the correlation between users 1648 and 5136 is 0.40298, and the correlation between users 918 and 2824 is -0.31706.

In [11]:
corr_df.loc[1648, 5136]

0.40298018845699629

In [12]:
corr_df.loc[918, 2824]

-0.31706324373711381

## Identify the top 5 neighbors (the users with the 5 largest, positive correlations) for users 3867 and 89.

In [44]:
def top_n_corr(user, n, exclude_user=True):
    
    user_corr = corr_df.loc[user]
    
    if exclude_user:
        
        return user_corr.nlargest(n+1).drop(user)
    else:
        
        return user_corr.nlargest(n)

In [45]:
top_n_corr(3867, 5)

2492    0.476683
3853    0.464110
2486    0.438992
3712    0.400275
2288    0.379856
Name: 3867, dtype: float64

In [46]:
top_n_corr(89, 5)

4809    0.668516
5136    0.562449
860     0.539066
5062    0.525990
3525    0.475495
Name: 89, dtype: float64

For example, if the target user were #3712, the closest neighbors are 2824 (corr: 0.46291), 3867 (corr: 0.400275), 5062 (corr: 0.247693), 442 (corr: 0.22713), and 3853 (corr: 0.19366)

In [47]:
top_n_corr(3712, 5)

2824    0.462910
3867    0.400275
5062    0.247693
442     0.227130
3853    0.193660
Name: 3712, dtype: float64

# Create a new worksheet in your spreadsheet, and use it to compute the predictions for each movie for users 3867 and 89 by taking the correlation-weighted average of the ratings of the top-five neighbors (for each target user) for each movie.

$$\frac{\sum_{n=1}^{5}r_nw_n}{\sum_{n=1}^{5}w_n}$$

In [256]:
def predict_user_n(user, n_neighbor):
    
    neighbor_corr = top_n_corr(user, n_neighbor, exclude_user=True)
    
    # numerator
    numerator = df.fillna(0).multiply(neighbor_corr).sum(axis=1)
    
    # denominator
    # tem jeito melhor?!
    denominator = (~df.isnull()).multiply(neighbor_corr).sum(axis=1)
    denominator = denominator.where(denominator != 0, np.inf)
    
    return numerator / denominator

For example, if the user ID was 3712, the correct submission would be:

Top Movie: 641 Prediction: 5.000

2nd Movie: 603 Prediction: 4.856

3rd Movie: 105 Prediction: 4.739

And if the user ID were 3525, there would be a three-way tie among:

Movies: 238, 194, and 38 (all with a prediction of 5.000) 

In [190]:
predict_user_n(3712, 5).nlargest(3)

641: Requiem for a Dream (2000)    5.000000
603: The Matrix (1999)             4.855924
105: Back to the Future (1985)     4.739173
dtype: float64

And if the user ID were 3525, there would be a three-way tie among:

Movies: 238, 194, and 38 (all with a prediction of 5.000) 

In [191]:
predict_user_n(3525, 5).nlargest(3)

38: Eternal Sunshine of the Spotless Mind (2004)    5.0
194: Amelie (2001)                                  5.0
238: The Godfather (1972)                           5.0
dtype: float64

In [252]:
predict_user_n(3867, 5).nlargest(3)

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    4.760291
155: The Dark Knight (2008)                                    4.551454
122: The Lord of the Rings: The Return of the King (2003)      4.507637
dtype: float64

In [253]:
predict_user_n(89, 5).nlargest(3)

238: The Godfather (1972)               4.894124
278: The Shawshank Redemption (1994)    4.882194
807: Seven (a.k.a. Se7en) (1995)        4.774093
dtype: float64

# Part 2 - Normalization

In [257]:
def predict_user_n_norm(user, n_neighbor):
    
    neighbor_corr = top_n_corr(user, n_neighbor, exclude_user=True)
    
    # numerator
    # rating - média do rating p/ o user
    rn_ = df.subtract(df.mean())
    
    numerator = rn_.fillna(0).multiply(neighbor_corr).sum(axis=1)
    
    # numerator
    # tem jeito melhor?!
    denominator = (~df.isnull()).multiply(neighbor_corr).sum(axis=1)
    denominator = denominator.where(denominator != 0, np.inf)
    
    return df.loc[:, user].mean() + numerator / denominator

For example, if the user ID was 3712, the correct submission would be:

Top Movie: 641 Prediction: 5.900

2nd Movie: 603 Prediction: 5.546

3rd Movie: 105 Prediction: 5.501

In [258]:
predict_user_n_norm(3712, 5).nlargest(3)

641: Requiem for a Dream (2000)    5.900000
603: The Matrix (1999)             5.545567
105: Back to the Future (1985)     5.500585
dtype: float64

And if the user ID were 3525, the top three would be:

Movies: 238, 194, and 38 (all with a prediction of 5.000)

Top Movie: 238 Prediction: 4.760

2nd Movie: 424 Prediction: 4.663

3rd Movie: 134 Prediction: 4.585

In [250]:
predict_user_n_norm(3525, 5).nlargest(3)

238: The Godfather (1972)                4.759504
424: Schindler's List (1993)             4.663251
134: O Brother Where Art Thou? (2000)    4.585337
dtype: float64

In [254]:
predict_user_n_norm(3867, 5).nlargest(3)

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    5.245509
155: The Dark Knight (2008)                                    4.856770
77: Memento (2000)                                             4.777803
dtype: float64

In [255]:
predict_user_n_norm(89, 5).nlargest(3)

238: The Godfather (1972)               5.322015
278: The Shawshank Redemption (1994)    5.261424
275: Fargo (1996)                       5.241111
dtype: float64